In [ ]:
import os
import torch
import pandas as pd
import import_ipynb
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from typing import List
import multiprocessing

from halfpipe.file_index.bids import BIDSIndex

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
class SkullstripDataset(Dataset):
    """Skullstrip reports Dataset"""
    
    def __init__(self, usable_dir, labels, transform=None):
        """
        Args:
            root_dir: directory containing t1w and mask files
            labels(iterable): our labels for the images usable/unusable only return label or patient name as well?
            transform: optional transformations on images
        """
        self.usable_dir = usable_dir
        self.labels = labels
        self.transform = transform
        
        bids_idx = BIDSIndex()
        bids_idx.put(usable_dir)
        
        subjects = bids_idx.get_tag_values("sub")
        for subject in subjects:
            t1w = bids_idx.get(sub=subject, suffix="T1w")
            if t1w is None:
                continue
            mask = bids_idx.get(sub=subject, suffix="mask")
            if mask is None:
                continue
            (cur_t1w,) = t1w
            (cur_mask,) = mask
            
    
    def __len__(self):
        return len(self.root_dir) # or len of self.skullstrips
    
    def __getitem__(self,idx):
        cur_skullstrip = skullstrips[idx]
        
        if self.transform is not None:
            cur_skullstrip = self.transform
        
        return

def build_DataLoaders(train_dataset, val_dataset):
    """Build DataLoaders from the datasets."""
    train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True, num_workers=multiprocessing.cpu_count(), pin_memory=torch.cuda.is_available())
    val_loader = DataLoader(val_dataset, batch_size=5, shuffle=True, num_workers=multiprocessing.cpu_count(), pin_memory=torch.cuda.is_available())
    return train_loader, val_loader